# Fantasy Hockey Analysis

Trying to find some insights for my fantasy hockey league.

Note: Basic statistics data is from Rotowire. Advanced statistics data is from Natural Stat Trick.

Note 2: Ignoring goalies (for now).

Notes for me: 
1. Positions from Rotowire (and probably Natural Stat Trick) are out of date as the stats are from last year.
2. First names for some players may be slightly different between the two websites (i.e. Matt vs. Matthew).

## Table of Contents

1. [League Settings](#League-Settings)
2. [Importing Python Libraries](#Importing-Python-Libraries)
3. [Reading and Cleaning the Data](#Reading-and-Cleaning-the-Data)
4. [Add Fantasy Points Column](#Add-Fantasy-Points-Column)
5. [Value Over Replacement](#Value-Over-Replacement)
  1. [Total VORP](#Total-VORP)
  2. [VORP per Game](#VORP-per-Game)
6. [Projected Fantasy Points](#Projected-Fantasy-Points)
6. [Advanced Stats](#Advanced-Stats)
  1. [Reading the Advanced Stats Data](#Reading-the-Advanced-Stats-Data)
  2. [Merge and Drop Columns](#Merge-and-Drop-Columns)

## League Settings

Scoring for my points league.

In [1]:
points_per_goal = 3
points_per_assist = 2
points_per_ppp = 1
points_per_plus_minus = 1
points_per_shot = 0.4
points_per_pim = 0.5
points_per_hit = 0.2
points_per_blk = 0.2

Number of teams and players

In [2]:
num_of_teams = 10
num_of_c = 2
num_of_rw = 2
num_of_lw = 2
num_of_d = 4
num_of_util = 1
num_of_g = 2
num_of_bn = 5

Cutoff to filter out players so that we don't have really weird averages because of small sample size

In [3]:
min_games = 20

Keeper list. Will hopefully have a better way of inputting this in the future.

In [4]:
keeper_list = ['Alex Ovechkin', 'David Pastrnak', 'Erik Karlsson', # Team 1
               'Patrick Kane', 'Auston Matthews', 'Vladimir Tarasenko', # Team 2
               'Nikita Kucherov', 'John Tavares', 'Jack Eichel', # Team 3
               'Brad Marchand', 'Steven Stamkos', 'Artemi Panarin', # Team 4
               'Johnny Gaudreau', 'Tyler Seguin', # Sergei Bobrovsky, # Team 5
               'Nathan MacKinnon', 'John Carlson', 'Thomas Chabot', # Team 6
               'Connor McDavid', 'Mitch Marner', # Andrei Vasilevsky, # Team 7
               'Leon Draisaitl', 'Mark Scheifele', 'Patrik Laine', # Team 8
               'Brent Burns', 'Aleksander Barkov', 'Gabriel Landeskog', # Team 9 - projected
               'Sidney Crosby', 'Mikko Rantanen', # Carey Price, # Team 10 - projected, probably not accurate.
              ]
keeper_list = []

## Importing Python Libraries

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

## Reading and Cleaning the Data

In [7]:
path = '../data/'

In [8]:
df = pd.read_csv(path + '2018-2019 Fantasy Hockey Stats.csv', skiprows=1, index_col='Player Name')
df.rename(columns={'G.1':'PPG', 'A.1':'PPA', 'G.2':'SHG', 'A.2':'SHA'}, inplace=True)
df['PPP'] = df['PPG'] + df['PPA']
df['SHP'] = df['SHG'] + df['SHA']
display(df.shape)
display(df.head())

(906, 18)

,Team,Pos,Games,G,A,Pts,+/-,PIM,SOG,GWG,PPG,PPA,SHG,SHA,Hits,BS,PPP,SHP
Player Name,,,,,,,,,,,,,,,,,,
Justin Abdelkader,DET,LW,71,6,13,19,-14,38,95,0,1,1,1,0,185,34,2,1
Pontus Aberg,TOR,LW,59,12,13,25,-14,20,101,1,3,4,0,0,45,11,7,0
Vitali Abramov,OTT,RW,1,0,0,0,-3,0,0,0,0,0,0,0,0,1,0,0
Noel Acciari,FLA,C,72,6,8,14,-3,47,99,2,0,0,0,0,221,36,0,0
Kenny Agostino,TOR,LW,63,6,18,24,-3,34,79,0,0,5,0,0,143,14,5,0


Filter out players below min games

In [9]:
df = df[df['Games']>=min_games]

Filter out keepers. Will make optional in future.

In [10]:
df.drop(keeper_list, axis=0, inplace=True)

## Add Fantasy Points Column

First need a function to calculate the fantasy points for a row (player).

In [11]:
def calculate_fantasy_score(row):
    total_points = (row['G'] * points_per_goal + 
                    row['A'] * points_per_assist +
                    row['PPP'] * points_per_ppp +
                    row['+/-'] * points_per_plus_minus +
                    row['SOG'] * points_per_shot +
                    row['PIM'] * points_per_pim +
                    row['Hits'] * points_per_hit +
                    row['BS'] * points_per_blk)
    return total_points

In [12]:
df['Fantasy Points'] = df.apply(calculate_fantasy_score, axis=1)
df['Avg. Fantasy Points'] = df['Fantasy Points']/df['Games']
df.head()

,Team,Pos,Games,G,A,Pts,+/-,PIM,SOG,GWG,PPG,PPA,SHG,SHA,Hits,BS,PPP,SHP,Fantasy Points,Avg. Fantasy Points
Player Name,,,,,,,,,,,,,,,,,,,,
Justin Abdelkader,DET,LW,71,6,13,19,-14,38,95,0,1,1,1,0,185,34,2,1,132.8,1.870423
Pontus Aberg,TOR,LW,59,12,13,25,-14,20,101,1,3,4,0,0,45,11,7,0,116.6,1.976271
Noel Acciari,FLA,C,72,6,8,14,-3,47,99,2,0,0,0,0,221,36,0,0,145.5,2.020833
Kenny Agostino,TOR,LW,63,6,18,24,-3,34,79,0,0,5,0,0,143,14,5,0,136.0,2.158730
Sebastian Aho,CAR,LW,82,30,53,83,25,26,243,7,3,21,4,1,65,34,24,5,375.0,4.573171


In [13]:
df[(df['Games']>=20) 
   & (df['Pos']=='D')
  ].sort_values('Avg. Fantasy Points',ascending=False).round(1)

,Team,Pos,Games,G,A,Pts,+/-,PIM,SOG,GWG,PPG,PPA,SHG,SHA,Hits,BS,PPP,SHP,Fantasy Points,Avg. Fantasy Points
Player Name,,,,,,,,,,,,,,,,,,,,
Mark Giordano,CGY,D,78,17,57,74,39,69,221,2,3,18,4,2,58,144,21,6,388.3,5.0
Brent Burns,SJ,D,82,16,67,83,13,34,300,6,7,21,1,4,90,105,28,5,399.0,4.9
Kris Letang,PIT,D,65,16,40,56,13,48,206,3,2,17,1,1,116,111,19,2,311.8,4.8
Dustin Byfuglien,WPG,D,42,4,27,31,4,69,115,1,1,15,0,0,77,51,16,0,192.1,4.6
John Carlson,WAS,D,80,13,57,70,21,34,185,1,3,30,0,0,54,164,33,0,341.6,4.3
Victor Hedman,TB,D,70,12,42,54,24,44,182,4,2,23,0,1,59,97,25,1,295.0,4.2
Alexander Edler,VAN,D,56,10,24,34,3,54,128,2,4,13,0,1,127,166,17,1,234.8,4.2
Erik Karlsson,SJ,D,53,3,42,45,6,22,169,1,2,18,0,0,44,69,20,0,220.2,4.2
Morgan Rielly,TOR,D,82,20,52,72,24,14,223,5,3,18,0,0,61,114,21,0,340.2,4.1


In [14]:
# df[(df['GP']>=20) & (df['Position']=='D')][['Position', 'GP', 'Shots','Fantasy Points', 'Avg. Fantasy Points']].sort_values('Avg. Fantasy Points',ascending=False).round(1)

## Value Over Replacement

As some positions are much thinner than others (D being weakest, C being strongest), let's try to compare them by creating a "Value Over Replacement" (or VORP) column.

Some assumptions to make this easier:
1. We'll ignore the bench for now as they can be anything. This means that if we have 10 teams with 2 centres, then replacement would be the 21st centre.
2. As centres are so deep, we'll consider utils to be centres as well (so this means replace for centre is the 31st centre, compared to 21st for RW and LW)
3. Although this isn't a problem in this dataset, we'll consider dual-eligible players to be 2 different players when calculating vorp. (Similarly, tri-eligible will be 3).
4. Dual- or tri-eligible players will have their vorp calculated based on their weakest position. Calculating the value of multi-position players is not obvious to me so this seems like a fair compromise.

Note: because of these assumptions, it might just be easier to do vorp for forwards vs. defense. However, I do want to highlight the positional scarcity of non-D positions (mainly LW)

Note 2: points 3 and 4 will be added later, when I get more accurate position data.

Note 3: After running, I feel like C's are a little over-valued. So let's just ignore utils for now (like bench)

### Total VORP

Let's first calculate VORP for total points.

In [15]:
def calculate_replacement_total_score_for_position(position):
    
    df_vorp = df[df['Pos']==position].sort_values('Fantasy Points', ascending=False)
    
    if position == 'C':
        index = num_of_teams * num_of_c
        return df_vorp['Fantasy Points'].iloc[index+6] # plus 6 for util - arbitrary
    
    elif position == 'LW':
        index = num_of_teams * num_of_lw
        return df_vorp['Fantasy Points'].iloc[index+2]
    
    elif position == 'RW':
        index = num_of_teams * num_of_rw
        return df_vorp['Fantasy Points'].iloc[index+2]
    
    else:
        #position == 'D'
        index = num_of_teams * num_of_d
        return df_vorp['Fantasy Points'].iloc[index]

In [16]:
calculate_replacement_total_score_for_position('C')

302.70000000000005

In [17]:
calculate_replacement_total_score_for_position('LW')

259.7

In [18]:
calculate_replacement_total_score_for_position('RW')

252.80000000000004

In [19]:
calculate_replacement_total_score_for_position('D')

212.3

Now let's add the column to df

In [20]:
df['Replacement'] = df['Pos'].apply(calculate_replacement_total_score_for_position)
df['VORP'] = df['Fantasy Points'] - df['Replacement']
df.drop('Replacement', axis=1, inplace=True)
df.head()

,Team,Pos,Games,G,A,Pts,+/-,PIM,SOG,GWG,PPG,PPA,SHG,SHA,Hits,BS,PPP,SHP,Fantasy Points,Avg. Fantasy Points,VORP
Player Name,,,,,,,,,,,,,,,,,,,,,
Justin Abdelkader,DET,LW,71,6,13,19,-14,38,95,0,1,1,1,0,185,34,2,1,132.8,1.870423,-126.9
Pontus Aberg,TOR,LW,59,12,13,25,-14,20,101,1,3,4,0,0,45,11,7,0,116.6,1.976271,-143.1
Noel Acciari,FLA,C,72,6,8,14,-3,47,99,2,0,0,0,0,221,36,0,0,145.5,2.020833,-157.2
Kenny Agostino,TOR,LW,63,6,18,24,-3,34,79,0,0,5,0,0,143,14,5,0,136.0,2.158730,-123.7
Sebastian Aho,CAR,LW,82,30,53,83,25,26,243,7,3,21,4,1,65,34,24,5,375.0,4.573171,115.3


In [21]:
# df.sort_values('VORP', ascending=False).round(2)

### VORP per Game

I generally prefer averages to totals so let's calculate average vorp. This will be based on each individual player's average fantasy points per game (as opposed to taking their total vorp and dividing by 82).

In [22]:
def calculate_replacement_avg_score_for_position(position):
    
    df_vorp = df[df['Pos']==position].sort_values('Avg. Fantasy Points', ascending=False)
    
    if position == 'C':
        index = num_of_teams * num_of_c
        return df_vorp['Avg. Fantasy Points'].iloc[index+6] # plus 6 for util - arbitrary
    
    elif position == 'LW':
        index = num_of_teams * num_of_lw
        return df_vorp['Avg. Fantasy Points'].iloc[index+2]
    
    elif position == 'RW':
        index = num_of_teams * num_of_rw
        return df_vorp['Avg. Fantasy Points'].iloc[index+2]
    
    else:
        #position == 'D'
        index = num_of_teams * num_of_d
        return df_vorp['Avg. Fantasy Points'].iloc[index]

In [23]:
calculate_replacement_avg_score_for_position('C')

3.9

In [24]:
calculate_replacement_avg_score_for_position('LW')

3.5859154929577466

In [25]:
calculate_replacement_avg_score_for_position('RW')

3.3658536585365852

In [26]:
calculate_replacement_avg_score_for_position('D')

2.81219512195122

So forwards are all pretty similar (keeping in mind extra centres). Replacement forward scores about 0.8 more per game than replacement D. This is without factoring in bench spots, which are generally 1 G, 1 D, 3 F

Now let's add the column to df

In [27]:
df['Replacement'] = df['Pos'].apply(calculate_replacement_avg_score_for_position)
df['Avg. VORP'] = df['Avg. Fantasy Points'] - df['Replacement']
df.drop('Replacement', axis=1, inplace=True)
df.head()

,Team,Pos,Games,G,A,Pts,+/-,PIM,SOG,GWG,PPG,PPA,SHG,SHA,Hits,BS,PPP,SHP,Fantasy Points,Avg. Fantasy Points,VORP,Avg. VORP
Player Name,,,,,,,,,,,,,,,,,,,,,,
Justin Abdelkader,DET,LW,71,6,13,19,-14,38,95,0,1,1,1,0,185,34,2,1,132.8,1.870423,-126.9,-1.715493
Pontus Aberg,TOR,LW,59,12,13,25,-14,20,101,1,3,4,0,0,45,11,7,0,116.6,1.976271,-143.1,-1.609644
Noel Acciari,FLA,C,72,6,8,14,-3,47,99,2,0,0,0,0,221,36,0,0,145.5,2.020833,-157.2,-1.879167
Kenny Agostino,TOR,LW,63,6,18,24,-3,34,79,0,0,5,0,0,143,14,5,0,136.0,2.158730,-123.7,-1.427185
Sebastian Aho,CAR,LW,82,30,53,83,25,26,243,7,3,21,4,1,65,34,24,5,375.0,4.573171,115.3,0.987255


In [28]:
df.sort_values('Avg. VORP', ascending=False).round(2)

,Team,Pos,Games,G,A,Pts,+/-,PIM,SOG,GWG,PPG,PPA,SHG,SHA,Hits,BS,PPP,SHP,Fantasy Points,Avg. Fantasy Points,VORP,Avg. VORP
Player Name,,,,,,,,,,,,,,,,,,,,,,
Nikita Kucherov,TB,RW,82,41,87,128,24,62,246,8,15,33,0,0,44,31,48,0,513.4,6.26,260.6,2.90
Alex Ovechkin,WAS,LW,81,51,38,89,7,40,338,5,18,10,0,0,223,41,28,0,472.0,5.83,212.3,2.24
Patrick Kane,CHI,RW,81,44,66,110,2,22,341,7,9,21,0,0,21,18,30,0,451.2,5.57,198.4,2.20
David Pastrnak,BOS,RW,66,38,43,81,6,32,235,4,17,16,0,0,57,25,33,0,365.4,5.54,112.6,2.17
Mark Giordano,CGY,D,78,17,57,74,39,69,221,2,3,18,4,2,58,144,21,6,388.3,4.98,176.0,2.17
Brent Burns,SJ,D,82,16,67,83,13,34,300,6,7,21,1,4,90,105,28,5,399.0,4.87,186.7,2.05
Kris Letang,PIT,D,65,16,40,56,13,48,206,3,2,17,1,1,116,111,19,2,311.8,4.80,99.5,1.98
Brad Marchand,BOS,LW,79,36,64,100,15,96,231,9,10,24,3,4,46,19,34,7,438.4,5.55,178.7,1.96
Nathan MacKinnon,COL,C,82,41,58,99,20,34,365,6,12,25,0,0,54,31,37,0,476.0,5.80,173.3,1.90


This seems fair. Maybe a little high on D.

## Projected Fantasy Points

As my projections are not finished yet, going to use Scott Cullen's for now

In [29]:
keeper_list = ['Alex Ovechkin', 'David Pastrnak', 'Erik Karlsson', # Team 1
               'Patrick Kane', 'Auston Matthews', 'Vladimir Tarasenko', # Team 2
               'Nikita Kucherov', 'John Tavares', 'Jack Eichel', # Team 3
               'Brad Marchand', 'Steven Stamkos', 'Artemi Panarin', # Team 4
               'Johnny Gaudreau', 'Tyler Seguin', # Sergei Bobrovsky, # Team 5
               'Nathan MacKinnon', 'John Carlson', 'Thomas Chabot', # Team 6
               'Connor McDavid', 'Mitchell Marner', # Andrei Vasilevsky, # Team 7
               'Leon Draisaitl', 'Mark Scheifele', 'Patrik Laine', # Team 8
               'Brent Burns', 'Aleksander Barkov', 'Gabriel Landeskog', # Team 9 - projected
               'Sidney Crosby', 'Mikko Rantanen', # Carey Price, # Team 10 - projected, probably not accurate.
              ]
#keeper_list = []

In [30]:
df_proj = pd.read_csv(path + '2019-2020 Cullen Projections.csv')
df_proj.rename(columns={'Name':'Player',
                        'HIT':'Hits',
                        'BLOCKS':'BS'}, inplace=True)
df_proj.set_index('Player', inplace=True)
df_proj.drop(keeper_list, axis=0, inplace=True)
df_proj = pd.merge(df_proj, df[['Pos']], how='left', left_index=True, right_index=True)
display(df_proj.shape)
display(df_proj.head())

(374, 10)

,G,A,Pts,PIM,PPP,SOG,Hits,BS,+/-,Pos
Player,,,,,,,,,,
Blake Wheeler,25,62,87,54,32,242,89,55,4,RW
Sebastian Aho,32,48,80,30,23,260,71,29,9,LW
Jonathan Huberdeau,27,52,79,36,26,215,49,43,-3,LW
Evgeni Malkin,28,50,78,86,28,201,49,31,-4,C
Claude Giroux,23,55,78,26,28,213,38,22,8,C


In [31]:
df_proj['Fantasy Points'] = df_proj.apply(calculate_fantasy_score, axis=1)
df_proj.head()

,G,A,Pts,PIM,PPP,SOG,Hits,BS,+/-,Pos,Fantasy Points
Player,,,,,,,,,,,
Blake Wheeler,25,62,87,54,32,242,89,55,4,RW,387.6
Sebastian Aho,32,48,80,30,23,260,71,29,9,LW,363.0
Jonathan Huberdeau,27,52,79,36,26,215,49,43,-3,LW,330.4
Evgeni Malkin,28,50,78,86,28,201,49,31,-4,C,347.4
Claude Giroux,23,55,78,26,28,213,38,22,8,C,325.2


In [32]:
df_proj.sort_values('Fantasy Points', ascending=False)

,G,A,Pts,PIM,PPP,SOG,Hits,BS,+/-,Pos,Fantasy Points
Player,,,,,,,,,,,
Blake Wheeler,25,62,87,54,32,242,89,55,4,RW,387.6
Sebastian Aho,32,48,80,30,23,260,71,29,9,LW,363.0
Evgeni Malkin,28,50,78,86,28,201,49,31,-4,C,347.4
Timo Meier,31,31,62,66,9,261,127,55,5,LW,342.8
Evander Kane,28,23,51,125,10,276,153,28,-8,LW,341.1
Taylor Hall,26,48,74,32,25,242,55,48,4,LW,336.4
Brayden Point,32,43,75,26,28,204,28,50,12,C,332.2
Jamie Benn,29,34,63,58,18,205,129,63,9,C,331.4
Jonathan Huberdeau,27,52,79,36,26,215,49,43,-3,LW,330.4


In [33]:
# df_proj[df_proj['Pos']=='D'].sort_values('Fantasy Points', ascending=False)

## Advanced Stats

Decided to keep this for later, though it's main use is for projections.

### Reading the Advanced Stats Data

Note: Using 2018-2019 All Strengths data.

In [34]:
# df_bio = pd.read_csv(path + 'Player Bios (as of 2018-2019).csv', index_col='Player')
# df_bio.drop('Unnamed: 0', axis=1, inplace=True)
# #df_bio['Date of Birth'] = pd.datetime(df_bio['Date of Birth'])
# display(df_bio.shape)
# display(df_bio.head())

In [35]:
# df_on_ice = pd.read_csv(path + '2018-2019 All Strengths On-Ice.csv', index_col='Player')
# df_on_ice.drop('Unnamed: 0', axis=1, inplace=True)
# display(df_on_ice.shape)
# display(df_on_ice.head())

In [36]:
# print(df_on_ice.columns.tolist())

Note: Some of the column names for this csv file have weird characters add it. For now, I'll just rename the columns but should look for better way.

In [37]:
# df_on_ice.rename(columns={'Off.\xa0Zone Starts':'Off. Zone Starts',
#                           'Neu.\xa0Zone Starts':'Neu. Zone Starts',
#                           'Def.\xa0Zone Starts':'Def. Zone Starts',
#                           'On\xa0The\xa0Fly Starts':'On The Fly Starts',
#                           'Off.\xa0Zone Start %':'Off. Zone Start %',
#                           'Off.\xa0Zone Faceoffs':'Off. Zone Faceoffs',
#                           'Neu.\xa0Zone Faceoffs':'Neu. Zone Faceoffs',
#                           'Def.\xa0Zone Faceoffs':'Def. Zone Faceoffs',
#                           'Off.\xa0Zone Faceoff %':'Off. Zone Faceoff %'}, inplace=True)
# display(df_on_ice.head(1))

In [38]:
# df_individual = pd.read_csv(path + '2018-2019 All Strengths Individual.csv', index_col='Player')
# df_individual.drop('Unnamed: 0', axis=1, inplace=True)
# display(df_individual.shape)
# display(df_individual.head())

### Merge and Drop Columns

Let's first determine which columns to keep. We're going to merge everything together so we will drop any repeated columns (df_individual will be our first dataframe so it will keep the columns).

Note: not going to include faceoffs (not relevant for my league).

In [39]:
# individual_cols_to_keep = ['Team', 'Position', 'GP', 'TOI', 'Goals', 'Total Assists',
#                            'First Assists', 'Second Assists', 'Total Points', 'IPP', 'Shots',
#                            'SH%', 'ixG', 'iCF', 'iFF', 'iSCF', 'iHDCF', 'Rush Attempts',
#                            'Rebounds Created', 'PIM', 'Total Penalties', 'Minor', 'Major',
#                            'Misconduct', 'Penalties Drawn', 'Giveaways', 'Takeaways', 'Hits',
#                            'Hits Taken', 'Shots Blocked']

In [40]:
# on_ice_cols_to_keep = ['CF', 'CA', 'CF%', 'FF', 'FA', 'FF%', 'SF', 'SA', 'SF%', 'GF', 'GA', 'GF%', 
#                        'xGF', 'xGA', 'xGF%', 'SCF', 'SCA', 'SCF%', 'HDCF', 'HDCA', 'HDCF%', 'HDGF', 'HDGA', 'HDGF%', 
#                        'MDCF', 'MDCA', 'MDCF%', 'MDGF', 'MDGA', 'MDGF%', 'LDCF', 'LDCA', 'LDCF%', 'LDGF', 'LDGA', 
#                        'LDGF%', 'On-Ice SH%', 'On-Ice SV%', 'PDO', 'Off. Zone Starts', 'Neu. Zone Starts', 
#                        'Def. Zone Starts', 'On The Fly Starts', 'Off. Zone Start %', 'Off. Zone Faceoffs',
#                        'Neu. Zone Faceoffs', 'Def. Zone Faceoffs', 'Off. Zone Faceoff %']

In [41]:
# bio_cols_to_keep = ['Age', 'Date of Birth', 'Birth City', 'Birth State/Province', 'Nationality', 
#                     'Height (in)', 'Weight (lbs)', 'Draft Year', 'Draft Team', 'Draft Round', 'Round Pick',
#                     'Overall Draft Position']

Let's merge!

In [42]:
# df = df_individual[individual_cols_to_keep].merge(df_on_ice[on_ice_cols_to_keep], 
#                                                     how='outer', left_index=True, right_index=True)
# df = df.merge(df_bio[bio_cols_to_keep], how='outer', left_index=True, right_index=True)
# #df = df.merge(df_plus_minus, how='left', left_index=True, right_index=True)
# display(df.shape)
# display(df.head())